Import the necessary python libraries and Susie objects.

In [1]:
%pip install -i https://test.pypi.org/simple/ susie==1.0.3

Looking in indexes: https://test.pypi.org/simple/
You should consider upgrading via the '/Users/maliabarker/Desktop/main/susie/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
from susie.transit_times import TransitTimes
from susie.ephemeris import Ephemeris
import matplotlib.pyplot as plt
from astropy import coordinates as coords
from astropy import units as u
from astropy import time

In [6]:
coors = ('06h30m32.79s +29d40m20.16s')
c = coords.SkyCoord(coors, unit=(u.hourangle, u.deg), obstime="J1992.21")
c.ra

<Longitude 97.636625 deg>

Add your transit data to the TransitTimes object.

In [10]:
# STEP 1: Upload data from file
filepath = "./malia_examples/WASP12b_transit_ephemeris.csv"
data = np.genfromtxt(filepath, delimiter=',', names=True)
# STEP 2: Break data up into epochs, mid transit times, and error
epochs = data["epoch"] - np.min(data["epoch"])
mid_transit_times = data["transit_time"] - np.min(data["transit_time"])
mid_transit_times_err = data["sigma_transit_time"]
# NOTE: You can use any method and/or file type to upload your data. Just make sure the resulting variables (epoch, mid transit times, and mid transit time errors) are numpy arrays
# STEP 2.5 (Optional): Make sure the epochs are integers and not floats
epochs = epochs.astype('int')
# STEP 3: Create new transit times object with above data
transit_times_obj1 = TransitTimes(epochs, mid_transit_times, mid_transit_times_err)
# STEP 4 (Optional): To check that the resulting data is correct, you can print each variable and the corresponding data using this command: print(vars(transit_times_obj1))

In [11]:
mid_tt = time.Time(mid_transit_times, format='jd', scale=None)

In [18]:
test_arr = np.ones_like(mid_tt, dtype=float)

In [19]:
test_tt = time.Time(test_arr, format='jd', scale=None)

In [20]:
np.all(test_tt.value == 1)

True

In [22]:
earth_center = coords.EarthLocation.from_geocentric(0., 0., 0., unit=u.m)

In [25]:
earth_center.value

(0., 0., 0.)

In [32]:
obs_location = coords.EarthLocation.from_geodetic(-116, 42)

In [38]:
obj_coord = coords.SkyCoord(ra=97.64, dec=29.67, unit='deg', frame='icrs')

In [48]:
print(obj_coord.ra.value)

97.64


Create the Ephemeris object and add your TransitTimes object.

In [ ]:
ephemeris_obj1 = Ephemeris(transit_times_obj1)

Fit your transit time data to an ephemeris model. You can specify what type of model with the options 'linear' or 'quadratic'.

In [ ]:
# Getting a linear model will solve for period and conjuction time (and their respective errors)
linear_model_data = ephemeris_obj1.get_model_ephemeris('linear')

In [ ]:
# Show model fit
plt.scatter(x=transit_times_obj1.epochs, y=linear_model_data, marker='v')
plt.scatter(x=transit_times_obj1.epochs, y=transit_times_obj1.mid_transit_times, marker='o')
plt.show()

In [ ]:
# Getting a quadratic model will solve for period, conjunction time, and period change by epoch (and their respective errors)
quad_model_data = ephemeris_obj1.get_model_ephemeris('quadratic')

In [ ]:
# Show model fit
plt.scatter(x=transit_times_obj1.epochs, y=quad_model_data, marker='v')
plt.scatter(x=transit_times_obj1.epochs, y=transit_times_obj1.mid_transit_times, marker='o')
plt.show()